In [1]:
import os
import sys
import time
import pandas as pd


In [2]:
output="experiment_2211"
T=200 #Training time period
C=1000000 #Capital available
file=1 #Index_file
m=8 #Inital Size of Kernel
lbuck=2 #Size Of Buckets
pho=0.1 #Transaction Cost Proportion
lamda=1/(1000*C) # lower_bound for capital invested in jth stock
nuh=0.8  # upper_bound
xii=1.2 # Limit for Tracking Error
k=14 #Cardinality constraint for stocks in portfolio
p=3 #If stock not selected in optimal soln in last p iters then it is to be dropped from kernel
f=12
#Rest parameters are defined inside underlying scripts

In [3]:
print ("+----------------------------------------------------+")
print ("    Step 1: Solving Linear Relaxation of EIT-Basic")
print ("+----------------------------------------------------+")
s=time.time()
#print ("len sys.argv in experiment={}".format(len(sys.argv)))
out=os.system("python ./src/linear_relaxation.py {} {} {} {} {} {} {} {} {} {}".format(\
    file,T,xii,k,pho,nuh,C,lamda,f,output))

text_file=open(output+"/EIT_LP_details.txt")
lines=text_file.readlines()
failure=bool(int(lines[0][-2]))
z_lp=float(lines[1].split("=")[-1][:-2])
result_lp=pd.read_csv(output+"/result_index_{}.csv".format(file))
print ("+----------------------------------------------------+")
print ("    Step 1 complete in {:.2f}s".format(time.time()-s))
print ("+----------------------------------------------------+")

+----------------------------------------------------+
    Step 1: Solving Linear Relaxation of EIT-Basic
+----------------------------------------------------+
+----------------------------------------------------+
    Step 1 complete in 2.12s
+----------------------------------------------------+


In [4]:
print ("+----------------------------------------------------+")
print ("    Step 2a: Sort Securities and create bucktes")
print ("+----------------------------------------------------+")
s=time.time()
from src.sort_and_buckets import *
# Create dummy problem using PULP
LP,q_T=dummy_problem(T,C,file)
q_T.drop("index",inplace=True)
objective=LP.objective

#Create ranked list and buckets
L=sort_securities(result_lp,q_T,objective,lamda,C) #Ranked List
kernel=L[:m]
initial_kernel=kernel.copy() #Create copy of Initial Kernel
buckets=create_buckets(L,m,lbuck)
Nb=len(buckets)
print ("+----------------------------------------------------+")
print ("    Step 2a complete in {:.2f}s".format(time.time()-s))
print ("+----------------------------------------------------+")

+----------------------------------------------------+
    Step 2a: Sort Securities and create bucktes
+----------------------------------------------------+
+----------------------------------------------------+
    Step 2a complete in 0.28s
+----------------------------------------------------+


/Users/ashish1610dhiman/opt/anaconda3/envs/eit_paper/lib/python3.8/site-packages/pulp/pulp.py:1198: UserWarning: Spaces are not permitted in the name. Converted to '_'
  warnings.warn("Spaces are not permitted in the name. Converted to '_'")
/Users/ashish1610dhiman/data_projects/index_tracking_kgp/eit_paper/src/sort_and_buckets.py:34: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  not_optimal.sort_values(by="value",ascending=False,inplace=True)


In [5]:
print ("+----------------------------------------------------+")
print ("    Step 2b: Sort Securities and create bucktes")
print ("+----------------------------------------------------+")
s=time.time()
from src.EIT_kernel import *
try:
    status,z=EIT_kernel(kernel,C,T,file,lamda,nuh,xii,k,pho,f,output)
    failure=bool(status.value>0)
except:
    print ("ERROR in EIT Kernel")
    print (status)

execution_result=pd.DataFrame()
temp=pd.DataFrame()
temp["bucket"]=[0]
temp["kernel_size"]=[len(kernel)]
temp["problem_status"]=[status]
temp["z_value"]=[z]
execution_result=execution_result.append(temp,ignore_index=True)
result_kernel=pd.read_csv(output+"/EIT_kernel_result_index_{}.csv".format(file))
plot_results(kernel,result_kernel,file,T,output);
print ("+----------------------------------------------------+")
print ("    Step 2b complete in {:.2f}s".format(time.time()-s))
print ("+----------------------------------------------------+")

+----------------------------------------------------+
    Step 2b: Sort Securities and create bucktes
+----------------------------------------------------+
Solving EIT(kernel)
***************************************************
***************************************************

Optimisation Status=0,Objective Value=1300.828
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
+----------------------------------------------------+
    Step 2b complete in 0.98s
+----------------------------------------------------+


In [6]:
print ("+----------------------------------------------------+")
print ("    Step 3: Sort Securities and create bucktes")
print ("+----------------------------------------------------+")
s=time.time()
from src.EIT_bucket import *
"""Initialise p_dict and z_low"""
p_dict={}
for stock in L:
    p_dict[stock]=0
z_low=z

kernel_record=[]
eit_model_record=[]
for i in range(1,Nb+1):
    bucket=buckets[i]
    #Add bucket to kernel
    kernel_copy=kernel.copy()
    kernel_record.append(kernel_copy)
    #kernel=kernel+bucket
    print ("\n\nFor bucket={}".format(str(i)))
    #Solve EIT(K+Bi)
    try:
        status,z,selected,EIT_model=EIT_bucket(kernel,bucket,i,failure,\
                                               z_low,C,T,file,lamda,nuh,xii,k,pho,f,output)
        eit_model_record.append(EIT_model)
    except:
        print ("Error in this bucket")
        continue
    if status.value==0: #Check if EIT(kernel+bucket) is feasible
        if failure==True: #check if EIT(Kernel) was in-feasible
            failure=False
        #Update lower_bound
        print ("Updating Lower Bound")
        if(z>z_low):
            z_low=z
        """Update Kernel"""
        #Add stocks from bucket which are selected in optimal
        print ("Updating Kernel")
        print ("Length of Old Kernel={}".format(len(kernel_copy)))
        kernel=kernel_copy+selected
        print ("Length of Updated Kernel={}".format(len(kernel)))
        #Make p=0 if stock just selected in Kernel
        for stock in selected:
            p_dict[stock]=0
        #Update p_dict
        result_bucket=pd.read_csv(output+"/EIT_bucket_{}_result_index_{}.csv".format(i,file))
        plot_results(kernel_copy,bucket,i,result_bucket,file,T,output)
        result_bucket.index=result_bucket["security"]
        result_bucket.drop(["security"],axis=1,inplace=True)
        for stock in kernel:
            if (result_bucket.loc[stock]['y']==0):
                p_dict[stock]+=1 #Increase by 1 if not selected in optimal
        #Remove from Kernel
        to_remove=[stock for (stock,p_value) in p_dict.items() if p_value > p]
        for stock in to_remove:
            if stock in kernel:
                print ("Removing {} from kernel".format(stock))
                kernel.remove(stock)
                p_dict[stock]=0
        print ("Current Length Kernel={}".format(len(kernel)))
    else:
        kernel=kernel_copy
    temp=pd.DataFrame()
    temp["bucket"]=[i]
    temp["kernel_size"]=[len(kernel)]
    temp["problem_status"]=[status]
    temp["z_value"]=[z]
    execution_result=execution_result.append(temp,ignore_index=True)
print ("+----------------------------------------------------+")
print ("    Step 3 complete in {:.2f}s".format(time.time()-s))
print ("+----------------------------------------------------+")

+----------------------------------------------------+
    Step 3: Sort Securities and create bucktes
+----------------------------------------------------+


For bucket=1
Solving EIT(kernel+bucket-1)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1479.0305487388523
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_15
security_29
***************************************************
Updating Lower Bound
Updating Kernel
Length of Old Kernel=8
Length of Updated Kernel=10
Current Length Kernel=10


For bucket=2
Solving EIT(kernel+bucket-2)
***************************************************
Optimisation Status=OptimizationStatus.OPTIMAL,Objective Value=1589.7274557883093
OPTIMAL(0), ERROR(-1), INFEASIBLE(1), UNBOUNDED(2)
Following stocks from bucket added to kernel:
security_23
security_8
***************************************************
Updating Lower Bound
Updating

In [7]:
print ("+----------------------------------------------------+")
print ("BEST Objective from EIT:{:.3f}".format(z_low))
print ("BEST Objective from Linear Relaxation:{:.3f}".format(z_lp))
print ("+----------------------------------------------------+")

+----------------------------------------------------+
BEST Objective from EIT:1633.800
BEST Objective from Linear Relaxation:1653.500
+----------------------------------------------------+
